In [1]:
# Import required libraries                   
# Import required library
import os
# Import required library
import numpy as np                    
# Import required library
import pandas as pd
# Import required library
import torch
# Import required library
import torch.nn as nn
# Import required library
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.metrics import f1_score     
from PIL import Image   
# Import required library
import copy

In [2]:
# Set the device to CPU or GPU based on availability
# Set the device to use GPU if available, else fallback to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
# 2. Paths
train_df = pd.read_csv('/kaggle/input/soil-classification-part-2/soil_competition-2025/train_labels.csv')
test_df = pd.read_csv('/kaggle/input/soil-classification-part-2/soil_competition-2025/test_ids.csv')
train_dir = '/kaggle/input/soil-classification-part-2/soil_competition-2025/train'
test_dir = '/kaggle/input/soil-classification-part-2/soil_competition-2025/test'

In [4]:
# # 5. Custom Dataset

# class SoilDataset(Dataset):
#     def __init__(self, dataframe, img_dir, transform):
#         self.df = dataframe
#         self.img_dir = img_dir
#         self.transform = transform

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         image_id = self.df.iloc[idx,0]
#         image_path = os.path.join(self.img_dir, image_id)
#         image = Image.open(image_path).convert("RGB")
#         image = self.transform(image)
#         return image, image_id

# Define custom PyTorch Dataset class for loading soil images
# Creating Dataset class for loading the image dataset
# Define custom PyTorch Dataset class for loading soil images
class SoilDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, is_test=False):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_id = self.dataframe.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, image_id)
        image = Image.open(img_path).convert('RGB')  

        if self.transform:
            image = self.transform(image)

        if self.is_test:
            return image, image_id
        else:
            label = self.dataframe.iloc[idx, -1] 
            return image, label

In [5]:
initial_transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor()])
initial_dataset = SoilDataset(train_df, train_dir, initial_transform)
initial_loader = DataLoader(initial_dataset, batch_size=32, shuffle=False)

In [6]:
# Function to calculate the mean and standard deviation
def calculate_mean_std(dataloader):
    mean = 0
    std = 0
    total_images = 0
    for images, _ in dataloader:
        batch_size = images.size(0)
        images = images.view(batch_size, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images += batch_size
    mean /= total_images
    std = torch.sqrt(std / (total_images * images.size(2)))
    return mean,std

In [7]:
mean,std = calculate_mean_std(initial_loader)
print(f"mean:{mean}")
print(f"standard_deviation:{std}")

mean:tensor([0.5194, 0.4144, 0.3265])
standard_deviation:tensor([0.0017, 0.0016, 0.0016])


In [8]:
# Transforms for train data and test data
train_transform = transforms.Compose([
    transforms.Resize((260, 260)),                    # Resizing input size
    transforms.RandomHorizontalFlip(),                # Horizontal flips
    transforms.RandomRotation(15),                    # Random rotation
    transforms.ColorJitter(0.3, 0.3, 0.3),            # Color jitters
    transforms.ToTensor(),                            # Converting tensor
    transforms.Normalize(mean,std)                    # Normalize the data by using the mean and std 
])

test_transform = transforms.Compose([                 
    transforms.Resize((260, 260)),                    # Resizing input size
    transforms.ToTensor(),                            # Converting to tensor
    transforms.Normalize(mean,std)                    # Normalize the data by using the mean and std
])

In [9]:
train_dataset = SoilDataset(train_df, train_dir, train_transform)
test_dataset = SoilDataset(test_df, test_dir, test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [10]:
# Loading pretrained EfficientNet-B2 model
model = models.efficientnet_b2(pretrained=True)

# Modify the classification layer to classify 4 soil types
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
model = model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B2_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 197MB/s]


In [11]:
# Creating the loss function of CrossEntropy with label smoothening to avoid overfitting
criterion = nn.CrossEntropyLoss()

# Creating the optimizer
optimizer = optim.AdamW(model.parameters(), lr=0.0001)

# Creating scheduler to control the learning rate
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [12]:
# Training function to determine the best weights based on highest least f1 score
def train_model(model, train_loader, epochs=20):
    final_weights = None
    best_f1 = 0.0# Save the best minumum F1 score

    for epoch in range(epochs):
        print(f"Epoch no: {epoch}")
        model.train()
        running_loss = 0.0
        all_preds = []
        all_labels = []

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.item()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        # Calculate class-wise F1-scores
        all_f1 = f1_score(all_labels, all_preds, average=None)
        min_f1 = min(all_f1)
        print(f"Epoch:{epoch+1} Loss:{running_loss:.4f} Min F1:{min_f1:.4f}")

        # Step the LR scheduler
        scheduler.step()

        # Save the best model
        if min_f1 > best_f1:
            best_f1 = min_f1
            final_weights = model.state_dict()
            print("New model saved!")

    # Load best model before returning
    model.load_state_dict(final_weights)
    return model

In [13]:
# Epochs and training the model
model = train_model(model, train_loader, epochs=20)

Epoch no: 0
Epoch:1 Loss:10.3543 Min F1:0.0000
Epoch no: 1
Epoch:2 Loss:0.7723 Min F1:1.0000
New model saved!
Epoch no: 2
Epoch:3 Loss:0.2485 Min F1:1.0000
Epoch no: 3
Epoch:4 Loss:0.1537 Min F1:1.0000
Epoch no: 4
Epoch:5 Loss:0.0982 Min F1:1.0000
Epoch no: 5
Epoch:6 Loss:0.0846 Min F1:1.0000
Epoch no: 6
Epoch:7 Loss:0.0711 Min F1:1.0000
Epoch no: 7
Epoch:8 Loss:0.0640 Min F1:1.0000
Epoch no: 8
Epoch:9 Loss:0.0629 Min F1:1.0000
Epoch no: 9
Epoch:10 Loss:0.0504 Min F1:1.0000
Epoch no: 10
Epoch:11 Loss:0.0468 Min F1:1.0000
Epoch no: 11
Epoch:12 Loss:0.0435 Min F1:1.0000
Epoch no: 12
Epoch:13 Loss:0.0433 Min F1:1.0000
Epoch no: 13
Epoch:14 Loss:0.0411 Min F1:1.0000
Epoch no: 14
Epoch:15 Loss:0.0371 Min F1:1.0000
Epoch no: 15
Epoch:16 Loss:0.0355 Min F1:1.0000
Epoch no: 16
Epoch:17 Loss:0.0347 Min F1:1.0000
Epoch no: 17
Epoch:18 Loss:0.0336 Min F1:1.0000
Epoch no: 18
Epoch:19 Loss:0.0349 Min F1:1.0000
Epoch no: 19
Epoch:20 Loss:0.0327 Min F1:1.0000


In [14]:
# Evaluate model on test dataset
def predict_and_generate_submission(model):
    model.eval()
    predictions = []
    image_ids = []

    with torch.no_grad():
        for images, ids in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())
            image_ids.extend(ids)

# Map soil type names to numeric labels
    return pd.DataFrame({'image_id': image_ids, 'soil_type': predictions})

# Create submission.csv
submission = predict_and_generate_submission(model)
submission.to_csv('/kaggle/working/submission.csv', index=False)
print("submission generated")

submission generated
